In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Download original dataset
df = pd.read_csv(r'/Users/user/Downloads/WakePitchersAsOfJan2021original.csv', sep=",")
df

,Unnamed: 0,Unnamed: 1,Lead_Knee_Ang_Vel_Max,Lead_Knee_Ang_Vel_Max.1,Lead_Knee_Ang_Vel_Max.2,Back_Leg_GRF_mag_max,Hip Shoulders Sep@Footstrike,Hip Shoulders Sep@Footstrike.1,Hip Shoulders Sep@Footstrike.2,Lead_Knee_Angle@Footstrike,...,Max_Shoulder_Hor_Abd_Torque.1,Max_Shoulder_Hor_Add_Torque,Elbow_Torque@Max_Shoulder_Rot,Elbow_Torque@Release,Pitching_Elbow_Angle@Release.3,Pitching_Elbow_Angle@Release.4,Pitching_Elbow_Angle@Release.5,Pitching_Elbow_Angle_max,Pitching_Elbow_Angle_max.1,Pitching_Elbow_Angle_max.2
0,NaN,NaN,METRIC,METRIC,METRIC,METRIC,METRIC,METRIC,METRIC,METRIC,...,EVENT_LABEL,EVENT_LABEL,METRIC,METRIC,METRIC,METRIC,METRIC,METRIC,METRIC,METRIC
1,NaN,NaN,PROCESSED,PROCESSED,PROCESSED,PROCESSED,PROCESSED,PROCESSED,PROCESSED,PROCESSED,...,ORIGINAL,ORIGINAL,PROCESSED,PROCESSED,PROCESSED,PROCESSED,PROCESSED,PROCESSED,PROCESSED,PROCESSED
2,CSV_FILEPATH,ITEM,X,Y,Z,X,X,Y,Z,X,...,X,X,X,X,X,Y,Z,X,Y,Z
3,C:\Users\MAC_User3\Documents\Baseball\Data\Ant...,1,447.417145,115.749641,360.118927,1.699206,25.344067,7.582581,37.648571,29.001602,...,NaN,2.372,-0.00312,-0.01741,17.77682,-3.52921,-69.57805,109.66978,15.62593,-41.75812
4,C:\Users\MAC_User3\Documents\Baseball\Data\Ant...,1,442.906586,139.731674,324.02356,1.58675,7.837628,-12.884592,44.36829,42.986298,...,NaN,1.568,0.00041,-0.02035,14.81031,-4.94527,-65.53112,118.38982,16.82565,-33.91401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,F:\Baseball\Data\Will Andrews\Baseball Right-h...,1,425.348267,210.92865,285.726746,1.689309,12.767086,4.536453,63.951683,46.942848,...,NaN,3.69667,0.01147,-0.01325,25.91226,21.41368,-52.75742,120.5332,30.77501,-30.20061
370,F:\Baseball\Data\Will Fleming\Baseball Right-h...,1,476.96637,145.885223,267.351135,1.732077,12.781916,-0.655268,31.712303,48.794037,...,2.032,NaN,NaN,NaN,16.08992,-7.03863,77.15968,83.37768,0.49597,83.23711
371,F:\Baseball\Data\Will Fleming\Baseball Right-h...,1,472.78009,124.608902,305.572144,1.884336,13.166245,-4.100532,34.29747,51.383347,...,1.56,NaN,NaN,NaN,14.95552,-4.74176,84.83462,88.69725,3.70082,109.74193
372,F:\Baseball\Data\Will Fleming\Baseball Right-h...,1,491.521118,168.14621,239.631256,1.950241,8.84278,-3.583838,35.14053,51.705925,...,2.54,NaN,NaN,NaN,16.00492,-4.66646,83.89753,90.32365,3.53192,118.78202


In [3]:
print(df.columns)

Index(['Unnamed: 0', 'Unnamed: 1', 'Lead_Knee_Ang_Vel_Max',
       'Lead_Knee_Ang_Vel_Max.1', 'Lead_Knee_Ang_Vel_Max.2',
       'Back_Leg_GRF_mag_max', 'Hip Shoulders Sep@Footstrike',
       'Hip Shoulders Sep@Footstrike.1', 'Hip Shoulders Sep@Footstrike.2',
       'Lead_Knee_Angle@Footstrike', 'Lead_Knee_Angle@Footstrike.1',
       'Lead_Knee_Angle@Footstrike.2', 'Lead_Knee_Angle@Max_Shoulder_Rot',
       'Lead_Knee_Angle@Max_Shoulder_Rot.1',
       'Lead_Knee_Angle@Max_Shoulder_Rot.2', 'Lead_Knee_Angle@Release',
       'Lead_Knee_Angle@Release.1', 'Lead_Knee_Angle@Release.2',
       'Lead_Leg_GRF_mag_max', 'MASS', 'Max_Elbow_Flex_Torque',
       'Max_Elbow_Valgus_Torque', 'Max_Shoulder_Abd_Torque',
       'Max_Shoulder_Dist_Force', 'Max_Shoulder_Hor_Abd_Torque',
       'Max_Shoulder_IntRot_Torque', 'MaxTrunkLinearVel',
       'MaxTrunkLinearVel.1', 'MaxTrunkLinearVel.2', 'Pelvis_Angle@Footstrike',
       'Pelvis_Angle@Footstrike.1', 'Pelvis_Angle@Footstrike.2',
       'Pelvis_Angle@M

In [4]:
#Join XYZ row to Column names
list1 = df.iloc[2].values.tolist()
list2 = df.columns.tolist()

#Replaced @ symbols in Col names to _
list2 = [item.replace("@", "_") for item in list2]

#Adding underscore between Column names and XYZ label
underscore = '_'

list2 = [s + underscore for s in list2]

#Combining into new list
res = [i + j for i, j in zip(list2, list1)]

#Renaming original column names to new list of names
df.columns = res

#Delete everything in first column before "Data"
sep = 'Data'
df['Col'] = df['Unnamed: 0_CSV_FILEPATH'].str.split(sep, expand=True)[1]
df['Col']

#Split first column into Pitcher, Session Number, and Pitch Type columns
df[['Blank','Pitcher','MoCap Session','Pitch Type']] = df['Col'].str.split("\\", expand=True)
df.drop(columns=['Blank','Col','Unnamed: 0_CSV_FILEPATH'], axis=1, inplace=True)

#Drop first 3 rows, and then reset index
df.drop(df.index[0:3], inplace=True)
df = df.reset_index(drop=True)

#Move Pitcher, MoCap Session, Pitch Type to front of dataframe
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

df = movecol(df, 
             cols_to_move=['Pitcher','MoCap Session','Pitch Type','BALL_RELEASE_SPEED_mph_X'], 
             ref_col='Lead_Knee_Ang_Vel_Max_X',
             place='Before')

#Round Release Speed to 1 decimal
df['BALL_RELEASE_SPEED_mph_X'] = df['BALL_RELEASE_SPEED_mph_X'].astype(float)
df['BALL_RELEASE_SPEED_mph_X'].round(1)
df

,Unnamed: 1_ITEM,Pitcher,MoCap Session,Pitch Type,BALL_RELEASE_SPEED_mph_X,Lead_Knee_Ang_Vel_Max_X,Lead_Knee_Ang_Vel_Max.1_Y,Lead_Knee_Ang_Vel_Max.2_Z,Back_Leg_GRF_mag_max_X,Hip Shoulders Sep_Footstrike_X,...,Max_Shoulder_Hor_Abd_Torque.1_X,Max_Shoulder_Hor_Add_Torque_X,Elbow_Torque_Max_Shoulder_Rot_X,Elbow_Torque_Release_X,Pitching_Elbow_Angle_Release.3_X,Pitching_Elbow_Angle_Release.4_Y,Pitching_Elbow_Angle_Release.5_Z,Pitching_Elbow_Angle_max_X,Pitching_Elbow_Angle_max.1_Y,Pitching_Elbow_Angle_max.2_Z
0,1,Antonio Menendez,Baseball Right-handed,Breaking Ball RH 1.c3d,81.6,447.417145,115.749641,360.118927,1.699206,25.344067,...,NaN,2.372,-0.00312,-0.01741,17.77682,-3.52921,-69.57805,109.66978,15.62593,-41.75812
1,1,Antonio Menendez,Baseball Right-handed,Breaking Ball RH 2.c3d,78.0,442.906586,139.731674,324.02356,1.58675,7.837628,...,NaN,1.568,0.00041,-0.02035,14.81031,-4.94527,-65.53112,118.38982,16.82565,-33.91401
2,1,Antonio Menendez,Baseball Right-handed,Fastball RH 1.c3d,92.1,418.166321,129.898682,315.67627,1.762823,24.120113,...,NaN,1.84,0.00421,-0.00701,8.16356,0.89832,-75.91105,114.94846,18.45097,-54.36364
3,1,Antonio Menendez,Baseball Right-handed,Fastball RH 2.c3d,90.8,416.739197,128.615387,312.474976,1.650237,23.981058,...,NaN,3.252,-0.00055,-0.02147,17.69385,-5.32543,-83.72028,113.73364,17.98087,-62.50832
4,1,Antonio Menendez,Baseball Right-handed,Fastball RH 3.c3d,91.6,463.277435,106.045303,405.940674,1.695989,10.621145,...,NaN,3.344,0.00393,-0.01025,14.46912,-5.32881,-90.60345,121.48134,18.4576,-59.8975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,1,Will Andrews,Baseball Right-handed_3,Off Speed RH 3.c3d,80.3,425.348267,210.92865,285.726746,1.689309,12.767086,...,NaN,3.69667,0.01147,-0.01325,25.91226,21.41368,-52.75742,120.5332,30.77501,-30.20061
367,1,Will Fleming,Baseball Right-handed,Breaking Ball RH 1.c3d,81.7,476.96637,145.885223,267.351135,1.732077,12.781916,...,2.032,NaN,NaN,NaN,16.08992,-7.03863,77.15968,83.37768,0.49597,83.23711
368,1,Will Fleming,Baseball Right-handed,Fastball RH 2.c3d,88.8,472.78009,124.608902,305.572144,1.884336,13.166245,...,1.56,NaN,NaN,NaN,14.95552,-4.74176,84.83462,88.69725,3.70082,109.74193
369,1,Will Fleming,Baseball Right-handed,Fastball RH 3.c3d,90.5,491.521118,168.14621,239.631256,1.950241,8.84278,...,2.54,NaN,NaN,NaN,16.00492,-4.66646,83.89753,90.32365,3.53192,118.78202


In [5]:
df2 = pd.read_csv(r'/Users/user/Documents/BioMech Data/bullpen1.csv', sep=",")
df2

,PitchNo,Year,Date,Time,Pitcher,PitcherId,PitcherThrows,PitcherTeam,PitcherSet,TaggedPitchType,...,vz0,ax0,ay0,az0,PitchUID,CalibrationId,EffVelocity,PracticeType,Device,SpinEff
0,1,Fall_2020,8/27/20,14:17:42,"Teel, Austin",1000073457,Left,WAK_DEA,Stretch,Fastball,...,1.40947,8.94432,23.19294,-20.14064,3c8e64e6-5ef5-451f-a71d-730c65777355,4eec3286-491d-4280-8c70-19bd634277d4,83.46905,Pitch,19180023.0,93.759993
1,3,Fall_2020,8/27/20,14:20:14,"Teel, Austin",1000073457,Left,WAK_DEA,Stretch,Fastball,...,-0.69357,4.61011,28.21177,-16.78177,d3ac6174-ed41-42a1-ba09-768e2c2f2af0,4eec3286-491d-4280-8c70-19bd634277d4,86.08116,Pitch,19180023.0,96.200421
2,4,Fall_2020,8/27/20,14:21:01,"Teel, Austin",1000073457,Left,WAK_DEA,Stretch,ChangeUp,...,-0.13154,12.94546,24.06608,-23.28597,a2275eb1-7725-4d7a-853a-1dfca7c9186b,4eec3286-491d-4280-8c70-19bd634277d4,78.50002,Pitch,19180023.0,98.397516
3,5,Fall_2020,8/27/20,14:21:26,"Teel, Austin",1000073457,Left,WAK_DEA,Stretch,ChangeUp,...,-2.04514,8.49279,26.62147,-21.43577,bf270b1f-36ca-4be6-b7bc-ccc0a462aa09,4eec3286-491d-4280-8c70-19bd634277d4,77.70257,Pitch,19180023.0,97.831668
4,6,Fall_2020,8/27/20,14:21:48,"Teel, Austin",1000073457,Left,WAK_DEA,Stretch,ChangeUp,...,-3.10962,13.46111,27.17047,-28.43104,9df20451-2a48-4c9e-b6ee-19a521ebc51d,4eec3286-491d-4280-8c70-19bd634277d4,78.33630,Pitch,19180023.0,97.167404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,1,Winter_2021,2/3/21,16:07:12,"Fleming, William",1000017376,Right,WAK_DEA,Stretch,Sinker,...,-0.01819,-16.99891,29.30857,-25.34893,09784b6d-bf0d-4afd-bb14-8c518d3a750c,de766405-54c8-442b-825f-1af67f82fef1,NaN,Pitch,19180023.0,99.687724
2300,2,Winter_2021,2/3/21,16:07:29,"Fleming, William",1000017376,Right,WAK_DEA,Stretch,Sinker,...,-5.98565,-18.75657,28.54579,-20.73606,c9a9473f-22e0-4fbf-8beb-f9c3adaaf61f,de766405-54c8-442b-825f-1af67f82fef1,84.75560,Pitch,19180023.0,97.459442
2301,3,Winter_2021,2/3/21,16:08:03,"Fleming, William",1000017376,Right,WAK_DEA,Windup,Sinker,...,-1.49270,-19.22522,29.51512,-24.67760,ba32f60d-1070-49b0-a69f-5aaf73b3f4aa,de766405-54c8-442b-825f-1af67f82fef1,87.11417,Pitch,19180023.0,99.968496
2302,4,Winter_2021,2/3/21,16:08:58,"Fleming, William",1000017376,Right,WAK_DEA,Stretch,Sinker,...,-7.49190,-27.13191,29.43534,-10.55019,57dbfe82-0bbe-431a-b827-745a9756fe0a,de766405-54c8-442b-825f-1af67f82fef1,86.08009,Pitch,19180023.0,NaN


In [6]:
## Switch Last, First to First Last

df2[['last_name','first_name']] = df2['Pitcher'].loc[df2['Pitcher'].str.split().str.len() == 2].str.split(expand=True)
df2['last_name'] = df2['last_name'].str[:-1]
df2['PitcherName'] = df2['first_name'] + ' ' + df2['last_name']
df2 = df2.drop(columns=['first_name', 'last_name', 'Pitcher'])
df2['RelSpeed'].round(1)
df2

,PitchNo,Year,Date,Time,PitcherId,PitcherThrows,PitcherTeam,PitcherSet,TaggedPitchType,PitchSession,...,ax0,ay0,az0,PitchUID,CalibrationId,EffVelocity,PracticeType,Device,SpinEff,PitcherName
0,1,Fall_2020,8/27/20,14:17:42,1000073457,Left,WAK_DEA,Stretch,Fastball,Live,...,8.94432,23.19294,-20.14064,3c8e64e6-5ef5-451f-a71d-730c65777355,4eec3286-491d-4280-8c70-19bd634277d4,83.46905,Pitch,19180023.0,93.759993,Austin Teel
1,3,Fall_2020,8/27/20,14:20:14,1000073457,Left,WAK_DEA,Stretch,Fastball,Live,...,4.61011,28.21177,-16.78177,d3ac6174-ed41-42a1-ba09-768e2c2f2af0,4eec3286-491d-4280-8c70-19bd634277d4,86.08116,Pitch,19180023.0,96.200421,Austin Teel
2,4,Fall_2020,8/27/20,14:21:01,1000073457,Left,WAK_DEA,Stretch,ChangeUp,Live,...,12.94546,24.06608,-23.28597,a2275eb1-7725-4d7a-853a-1dfca7c9186b,4eec3286-491d-4280-8c70-19bd634277d4,78.50002,Pitch,19180023.0,98.397516,Austin Teel
3,5,Fall_2020,8/27/20,14:21:26,1000073457,Left,WAK_DEA,Stretch,ChangeUp,Live,...,8.49279,26.62147,-21.43577,bf270b1f-36ca-4be6-b7bc-ccc0a462aa09,4eec3286-491d-4280-8c70-19bd634277d4,77.70257,Pitch,19180023.0,97.831668,Austin Teel
4,6,Fall_2020,8/27/20,14:21:48,1000073457,Left,WAK_DEA,Stretch,ChangeUp,Live,...,13.46111,27.17047,-28.43104,9df20451-2a48-4c9e-b6ee-19a521ebc51d,4eec3286-491d-4280-8c70-19bd634277d4,78.33630,Pitch,19180023.0,97.167404,Austin Teel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,1,Winter_2021,2/3/21,16:07:12,1000017376,Right,WAK_DEA,Stretch,Sinker,Live,...,-16.99891,29.30857,-25.34893,09784b6d-bf0d-4afd-bb14-8c518d3a750c,de766405-54c8-442b-825f-1af67f82fef1,NaN,Pitch,19180023.0,99.687724,William Fleming
2300,2,Winter_2021,2/3/21,16:07:29,1000017376,Right,WAK_DEA,Stretch,Sinker,Live,...,-18.75657,28.54579,-20.73606,c9a9473f-22e0-4fbf-8beb-f9c3adaaf61f,de766405-54c8-442b-825f-1af67f82fef1,84.75560,Pitch,19180023.0,97.459442,William Fleming
2301,3,Winter_2021,2/3/21,16:08:03,1000017376,Right,WAK_DEA,Windup,Sinker,Live,...,-19.22522,29.51512,-24.67760,ba32f60d-1070-49b0-a69f-5aaf73b3f4aa,de766405-54c8-442b-825f-1af67f82fef1,87.11417,Pitch,19180023.0,99.968496,William Fleming
2302,4,Winter_2021,2/3/21,16:08:58,1000017376,Right,WAK_DEA,Stretch,Sinker,Live,...,-27.13191,29.43534,-10.55019,57dbfe82-0bbe-431a-b827-745a9756fe0a,de766405-54c8-442b-825f-1af67f82fef1,86.08009,Pitch,19180023.0,NaN,William Fleming


In [7]:
df3 = pd.read_csv(r'/Users/user/Documents/BioMech Data/DatesAddedWakePitchersJan2021.csv', sep=",")
df3['BALL_RELEASE_SPEED_mph_X'] = df['BALL_RELEASE_SPEED_mph_X'].astype(float)
df3['BALL_RELEASE_SPEED_mph_X'].round(1)
df3

,Unnamed: 0,Unnamed: 1_ITEM,Pitcher,Height,Weight,MoCap Session,Pitch Type,Date,BALL_RELEASE_SPEED_mph_X,PitchID,...,Max_Shoulder_Hor_Abd_Torque.1_X,Max_Shoulder_Hor_Add_Torque_X,Elbow_Torque_Max_Shoulder_Rot_X,Elbow_Torque_Release_X,Pitching_Elbow_Angle_Release.3_X,Pitching_Elbow_Angle_Release.4_Y,Pitching_Elbow_Angle_Release.5_Z,Pitching_Elbow_Angle_max_X,Pitching_Elbow_Angle_max.1_Y,Pitching_Elbow_Angle_max.2_Z
0,46,1,Antonio Menendez,77.0,225,Baseball Wake Forest Right-handed,Fastball RH WF 1.c3d,1/16/2019,81.6,NaN,...,2.084,NaN,NaN,NaN,36.95773,0.00000,66.97674,98.67232,0.00000,100.70495
1,47,1,Antonio Menendez,77.0,225,Baseball Wake Forest Right-handed,Fastball RH WF 2.c3d,1/16/2019,78.0,NaN,...,6.168,NaN,NaN,NaN,39.11061,0.00000,66.98484,106.69417,0.00001,110.35172
2,48,1,Antonio Menendez,77.0,225,Baseball Wake Forest Right-handed,Fastball RH WF 3.c3d,1/16/2019,92.1,NaN,...,0.872,NaN,NaN,NaN,42.61402,0.00000,42.89180,100.33922,0.00001,91.19677
3,75,1,Reese Robinson,78.5,235,Baseball Wake Forest Right-handed,Breaking Ball RH WF 1.c3d,5/29/2019,90.8,NaN,...,NaN,NaN,NaN,NaN,30.85772,0.00000,104.67638,89.96043,0.00001,128.45557
4,76,1,Reese Robinson,78.5,235,Baseball Wake Forest Right-handed,Breaking Ball RH WF 2.c3d,5/29/2019,91.6,NaN,...,NaN,NaN,NaN,NaN,34.25293,0.00000,100.90658,90.38155,0.00000,129.25749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,364,1,Will Andrews,70.0,147,Baseball Right-handed_3,Off Speed RH 1.c3d,1/15/2021,72.1,NaN,...,NaN,6.38333,0.00603,0.00740,26.75129,10.97013,-94.24082,127.40613,27.57376,-70.13182
360,365,1,Will Andrews,70.0,147,Baseball Right-handed_3,Off Speed RH 2.c3d,1/15/2021,71.4,NaN,...,NaN,3.13333,0.00808,0.00791,28.55533,12.28269,-100.79137,125.19627,26.72289,-71.68608
361,366,1,Will Andrews,70.0,147,Baseball Right-handed_3,Off Speed RH 3.c3d,1/15/2021,83.7,NaN,...,NaN,3.69667,0.01147,-0.01325,25.91226,21.41368,-52.75742,120.53320,30.77501,-30.20061
362,237,1,Edward McGraw,73.0,208,Baseball Right-handed_3,Breaking Ball RH 4.c3d,1/24/2021,83.9,NaN,...,NaN,4.56667,-0.00177,0.00274,17.67354,8.48768,-86.67651,113.43478,21.69525,-64.11304


In [10]:
df3.sort_values('BALL_RELEASE_SPEED_mph_X').drop_duplicates(subset=['Date', 'Pitcher'])
df3

,Unnamed: 0,Unnamed: 1_ITEM,Pitcher,Height,Weight,MoCap Session,Pitch Type,Date,BALL_RELEASE_SPEED_mph_X,PitchID,...,Max_Shoulder_Hor_Abd_Torque.1_X,Max_Shoulder_Hor_Add_Torque_X,Elbow_Torque_Max_Shoulder_Rot_X,Elbow_Torque_Release_X,Pitching_Elbow_Angle_Release.3_X,Pitching_Elbow_Angle_Release.4_Y,Pitching_Elbow_Angle_Release.5_Z,Pitching_Elbow_Angle_max_X,Pitching_Elbow_Angle_max.1_Y,Pitching_Elbow_Angle_max.2_Z
0,46,1,Antonio Menendez,77.0,225,Baseball Wake Forest Right-handed,Fastball RH WF 1.c3d,1/16/2019,81.6,NaN,...,2.084,NaN,NaN,NaN,36.95773,0.00000,66.97674,98.67232,0.00000,100.70495
1,47,1,Antonio Menendez,77.0,225,Baseball Wake Forest Right-handed,Fastball RH WF 2.c3d,1/16/2019,78.0,NaN,...,6.168,NaN,NaN,NaN,39.11061,0.00000,66.98484,106.69417,0.00001,110.35172
2,48,1,Antonio Menendez,77.0,225,Baseball Wake Forest Right-handed,Fastball RH WF 3.c3d,1/16/2019,92.1,NaN,...,0.872,NaN,NaN,NaN,42.61402,0.00000,42.89180,100.33922,0.00001,91.19677
3,75,1,Reese Robinson,78.5,235,Baseball Wake Forest Right-handed,Breaking Ball RH WF 1.c3d,5/29/2019,90.8,NaN,...,NaN,NaN,NaN,NaN,30.85772,0.00000,104.67638,89.96043,0.00001,128.45557
4,76,1,Reese Robinson,78.5,235,Baseball Wake Forest Right-handed,Breaking Ball RH WF 2.c3d,5/29/2019,91.6,NaN,...,NaN,NaN,NaN,NaN,34.25293,0.00000,100.90658,90.38155,0.00000,129.25749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,364,1,Will Andrews,70.0,147,Baseball Right-handed_3,Off Speed RH 1.c3d,1/15/2021,72.1,NaN,...,NaN,6.38333,0.00603,0.00740,26.75129,10.97013,-94.24082,127.40613,27.57376,-70.13182
360,365,1,Will Andrews,70.0,147,Baseball Right-handed_3,Off Speed RH 2.c3d,1/15/2021,71.4,NaN,...,NaN,3.13333,0.00808,0.00791,28.55533,12.28269,-100.79137,125.19627,26.72289,-71.68608
361,366,1,Will Andrews,70.0,147,Baseball Right-handed_3,Off Speed RH 3.c3d,1/15/2021,83.7,NaN,...,NaN,3.69667,0.01147,-0.01325,25.91226,21.41368,-52.75742,120.53320,30.77501,-30.20061
362,237,1,Edward McGraw,73.0,208,Baseball Right-handed_3,Breaking Ball RH 4.c3d,1/24/2021,83.9,NaN,...,NaN,4.56667,-0.00177,0.00274,17.67354,8.48768,-86.67651,113.43478,21.69525,-64.11304


In [21]:
df3['BALL_RELEASE_SPEED_mph_X'] = df3['BALL_RELEASE_SPEED_mph_X'].astype(float)
df3['Date'] = df3['Date'].astype(str)
df3['Pitcher'] = df3['Pitcher'].astype(str)

df2['RelSpeed'] = df2['RelSpeed'].astype(float)
df2['Date'] = df2['Date'].astype(str)
df2['PitcherName'] = df2['PitcherName'].astype(str)

In [22]:
new_df = pd.merge(df3, df2,  how='left', left_on=['Pitcher','Date','BALL_RELEASE_SPEED_mph_X'], right_on = ['PitcherName','Date','RelSpeed'])
new_df

,Unnamed: 0,Unnamed: 1_ITEM,Pitcher,Height,Weight,MoCap Session,Pitch Type,Date,BALL_RELEASE_SPEED_mph_X,PitchID,...,ax0,ay0,az0,PitchUID,CalibrationId,EffVelocity,PracticeType,Device,SpinEff,PitcherName
0,46,1,Antonio Menendez,77.0,225,Baseball Wake Forest Right-handed,Fastball RH WF 1.c3d,1/16/2019,81.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47,1,Antonio Menendez,77.0,225,Baseball Wake Forest Right-handed,Fastball RH WF 2.c3d,1/16/2019,78.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,48,1,Antonio Menendez,77.0,225,Baseball Wake Forest Right-handed,Fastball RH WF 3.c3d,1/16/2019,92.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,75,1,Reese Robinson,78.5,235,Baseball Wake Forest Right-handed,Breaking Ball RH WF 1.c3d,5/29/2019,90.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,76,1,Reese Robinson,78.5,235,Baseball Wake Forest Right-handed,Breaking Ball RH WF 2.c3d,5/29/2019,91.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,364,1,Will Andrews,70.0,147,Baseball Right-handed_3,Off Speed RH 1.c3d,1/15/2021,72.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,365,1,Will Andrews,70.0,147,Baseball Right-handed_3,Off Speed RH 2.c3d,1/15/2021,71.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,366,1,Will Andrews,70.0,147,Baseball Right-handed_3,Off Speed RH 3.c3d,1/15/2021,83.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,237,1,Edward McGraw,73.0,208,Baseball Right-handed_3,Breaking Ball RH 4.c3d,1/24/2021,83.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
new_df['RelSpeed']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
359   NaN
360   NaN
361   NaN
362   NaN
363   NaN
Name: RelSpeed, Length: 364, dtype: float64

In [ ]:
#df.to_csv(r'/Users/user/Documents/BioMech Data\Clean BioMech Data.csv')